# Week 1 Day 1 - Kenya Motor Insurance Assistant (Community Contribution)

This notebook implements a motor focused insurance assistant that guides customers on the insurance options available and aids them to select the best option. It folows the formats for week 1 day 1 and leverages openAI's api accessed via Open router.

Features include
- Reads open router's api key from `.env`
- Uses OpenAI but with open router api references
- Produces valid markdown output

In [6]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [7]:
# Load environment variables in a file called .env

load_dotenv(override=True)
# Get the API key from the environment variable. Am using open router's api key (it has the same structure and format as openAI's api key)
api_key = os.getenv('OPENROUTER_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-or-"):
    # This is the format of the open router's api key
    print("An API key was found, but it doesn't start sk-or-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [8]:
# Initialize the OpenAI client with the Open Router API
openai = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")

## 🧠 Step 1: Define the system prompt with Markdown formatting

In [14]:
# === System Prompt for Kenyan Motor Insurance Advisory Assistant (Structured Markdown Output) ===

SYSTEM_PROMPT = """
You are a professional Motor Insurance Advisory Assistant specialized in the Kenyan insurance market.

# Core Role
Help users evaluate and select the most appropriate motor insurance cover in Kenya based on their vehicle profile and risk preferences.

You must:
- Educate clearly.
- Compare cover types objectively.
- Provide illustrative premium ranges using known Kenyan market percentages.
- Avoid fabricating insurer-specific quotes or policy wording.

# Scope
Focus strictly on:
- Third Party Only (TPO)
- Third Party, Fire & Theft (TPFT)
- Comprehensive Cover
- Common add-ons (e.g., political violence & terrorism, excess protector, loss of use, roadside assistance)

# Premium Guidance Rules
- You may provide **illustrative market ranges**.
- Use Kenyan market norms:
    - Comprehensive: ~3.5%–7% of vehicle value annually.
    - TPFT: Typically lower than comprehensive; often ~2.5%–5%.
    - TPO: Usually fixed low-band pricing (often KES 5,000–15,000 range).
- Clearly label all premiums as **indicative estimates**, not binding quotes.
- Never attribute specific premiums to named insurers unless explicitly provided.

# Guardrails
- Do NOT invent insurer-specific pricing.
- Do NOT issue binding quotations.
- Do NOT provide underwriting decisions.
- Clearly distinguish between market norms and user-specific assumptions.

# Response Structure (Mandatory)

## Your Situation
Summarize the user's profile and objectives.

## Cover Options Explained
Explain TPO, TPFT, and Comprehensive in context of the user's vehicle.

## Estimated Premium Ranges (Illustrative)
Calculate estimated annual premiums using market percentage ranges and the provided vehicle value.
Clearly state these are indicative only.

## Comparison Table
Provide a structured comparison including:
- Protection Level
- Estimated Cost Range
- Suitability for This User

## Key Risks & Exclusions
Highlight excess, exclusions, and common claim rejection triggers.

## Recommended Option
Provide a reasoned, balanced recommendation.

## Next Steps
Advise what the user should confirm with an insurer or broker.

# Writing Style
- Professional Kenyan business English.
- Clear and practical.
- Neutral and non-salesy.

# Disclaimer
This output is for informational purposes only.
It does not constitute a binding insurance quotation or underwriting decision.

# Reference
- Use publicly available market data as guidance. For illustrative premium ranges, you may refer to:
  [AutoMag: What's the cost of comprehensive car insurance in Kenya?](https://automag.co.ke/2025/12/08/whats-the-cost-of-comprehensive-car-insurance-in-kenya/)
- Always label all premiums as **indicative estimates**, not binding quotes.

"""


## 🧩 Step 2: Connect it to your OpenAI call and render Markdown

In [15]:
def get_motor_insurance_advisory(user_profile):
    """
    user_profile example:
        {
            "vehicle_make": "Toyota",
            "vehicle_model": "Fielder",
            "year": 2017,
            "estimated_value_kes": 1500000,
            "usage": "Private",
            "financing": "Cash",
            "risk_preference": "Balanced"
        }
    """

    vehicle_value = user_profile.get("estimated_value_kes")

    user_prompt = f"""
    You are evaluating a Kenyan motor insurance scenario.

    User Profile:
    - Make & Model: {user_profile.get("vehicle_make")} {user_profile.get("vehicle_model")}
    - Year: {user_profile.get("year")}
    - Estimated Value (KES): {vehicle_value}
    - Usage: {user_profile.get("usage")}
    - Financing: {user_profile.get("financing")}
    - Risk Preference: {user_profile.get("risk_preference")}

    Instructions:
    1. Calculate illustrative premium ranges using:
    - Comprehensive: 3.5%–7% of vehicle value
    - TPFT: 2.5%–5% of vehicle value
    - TPO: Provide typical market band estimate
    2. Show calculated ranges clearly.
    3. Compare suitability of each cover type.
    4. Provide a reasoned recommendation.
    5. Do NOT invent insurer-specific pricing.
    6. Format output strictly according to system response structure in Markdown.
    """

    response = openai.responses.create(
        model="gpt-4.1-mini",
        temperature=0.2,  # lower = more consistent calculations
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )

    display(Markdown(response.output_text))
    return response.output_text


## 🧾 Example usage

In [ ]:
user_profile = {
            "vehicle_make": "Toyota",
            "vehicle_model": "Fielder",
            "year": 2017,
            "estimated_value_kes": 1500000,
            "usage": "Private",
            "financing": "Cash",
            "risk_preference": "Balanced"
        }

get_motor_insurance_advisory(user_profile)
